## RAG 

In [3]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="Dogs are great companions, known for their loyality and friendlyness.",
        metadata={"source":"mammal-pets-doc"}
        ),
    
    Document(
        page_content="Cats are independant pets that often enjoy their own space",
        metadata={"source":"mammal-pets-doc"}
        ),
]
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space')]

In [29]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
# llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x7fa70e54fe30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fa717faaab0>, root_client=<openai.OpenAI object at 0x7fa714120cb0>, root_async_client=<openai.AsyncOpenAI object at 0x7fa70ebed640>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [19]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

In [21]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore
vectorstore.similarity_search("cat")


[Document(id='5c7ff8df-bc9e-449a-9e7c-96f3a7b629d5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
 Document(id='a3357be0-f8f8-48dd-9fef-93a464fa5b4d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
 Document(id='a52cdb56-898e-4e4f-a64f-d171c132f0bc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.'),
 Document(id='a3f3d109-5d1a-4f78-9e88-2278869c040f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.')]

In [22]:
await vectorstore.asimilarity_search("cat")

[Document(id='5c7ff8df-bc9e-449a-9e7c-96f3a7b629d5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
 Document(id='a3357be0-f8f8-48dd-9fef-93a464fa5b4d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
 Document(id='a52cdb56-898e-4e4f-a64f-d171c132f0bc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.'),
 Document(id='a3f3d109-5d1a-4f78-9e88-2278869c040f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.')]

In [23]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='5c7ff8df-bc9e-449a-9e7c-96f3a7b629d5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
  1.2718664407730103),
 (Document(id='a3357be0-f8f8-48dd-9fef-93a464fa5b4d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space'),
  1.2751322984695435),
 (Document(id='a52cdb56-898e-4e4f-a64f-d171c132f0bc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.'),
  1.6283237934112549),
 (Document(id='a3f3d109-5d1a-4f78-9e88-2278869c040f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.'),
  1.630626916885376)]

In [24]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='5c7ff8df-bc9e-449a-9e7c-96f3a7b629d5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space')],
 [Document(id='a52cdb56-898e-4e4f-a64f-d171c132f0bc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.')]]

In [ ]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='5c7ff8df-bc9e-449a-9e7c-96f3a7b629d5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independant pets that often enjoy their own space')],
 [Document(id='a52cdb56-898e-4e4f-a64f-d171c132f0bc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendlyness.')]]

In [ ]:
#RAG


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.
{question}
Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("human",message)
])
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
